<a href="https://colab.research.google.com/github/Chali-Cheemala-Hema-Sri/Sentiment-Analyzer/blob/main/AI_Legal_Sentiment_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install gradio requests

In [3]:
import requests

API_KEY = "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"

def classify_sentiment(text):
    """
    Uses Mistral-7B from OpenRouter to classify legal sentiment.
    Returns: 'Positive', 'Neutral', or 'Negative'
    """
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    prompt = f"Analyze the sentiment of this legal document and reply with only one word: Positive, Neutral, or Negative.\n\n{text}"

    body = {
        "model": "mistralai/mistral-7b-instruct",
        "messages": [
            {"role": "system", "content": "You are a legal expert classifying sentiment in legal documents."},
            {"role": "user", "content": prompt}
        ]
    }

    try:
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", headers=headers, json=body)
        result = response.json()["choices"][0]["message"]["content"].strip()
        return result
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [4]:
sample_text = "The client expressed dissatisfaction with the legal advice provided."
print("Predicted Sentiment:", classify_sentiment(sample_text))


Predicted Sentiment: Negative


In [5]:
import json
from datetime import datetime
import os

def save_result(text, sentiment, filename="sentiment_results.json"):
    """
    Saves the input text and its predicted sentiment to a JSON file with a timestamp.
    """
    try:
        if os.path.exists(filename):
            with open(filename, "r") as f:
                data = json.load(f)
        else:
            data = []
    except json.JSONDecodeError:
        data = []

    data.append({
        "text": text[:200] + "..." if len(text) > 200 else text,
        "sentiment": sentiment,
        "timestamp": str(datetime.now())
    })

    with open(filename, "w") as f:
        json.dump(data, f, indent=2)

def load_results(filename="sentiment_results.json"):
    """
    Loads previously saved sentiment analysis results.
    """
    try:
        with open(filename, "r") as f:
            return json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        return []


In [6]:
sample_text = "The contract was terminated due to violation of multiple clauses."
sample_sentiment = classify_sentiment(sample_text)
save_result(sample_text, sample_sentiment)


In [7]:
saved_data = load_results()
print("✅ Saved Results Preview:")

for item in saved_data[-3:]:
    print(f"\n📝 Text: {item['text']}\n🔍 Sentiment: {item['sentiment']}\n📅 Time: {item['timestamp']}")


✅ Saved Results Preview:

📝 Text: The contract was terminated due to violation of multiple clauses.
🔍 Sentiment: Negative
📅 Time: 2025-07-08 08:52:13.624143


In [8]:
import gradio as gr

def analyze_and_save(text):
    sentiment = classify_sentiment(text)
    if not sentiment.startswith("❌"):
        save_result(text, sentiment)
    return sentiment

with gr.Blocks(title="⚖️ AI Legal Sentiment Analyzer") as demo:
    gr.Markdown("## ⚖️ AI Legal Sentiment Analyzer")
    gr.Markdown("Analyze legal text and classify sentiment as Positive, Neutral, or Negative.")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(
                label="📄 Legal Document Text",
                placeholder="Paste your legal paragraph here...",
                lines=8
            )
            analyze_button = gr.Button("🔍 Analyze Sentiment")

        with gr.Column():
            sentiment_output = gr.Textbox(label="🧠 Predicted Sentiment", interactive=False)

    with gr.Accordion("📂 Previous Analyses", open=False):
        saved_results = gr.JSON(label="📜 History")
        refresh_btn = gr.Button("🔄 Refresh History")
        refresh_btn.click(fn=load_results, outputs=saved_results)

    analyze_button.click(
        fn=analyze_and_save,
        inputs=text_input,
        outputs=sentiment_output
    ).then(
        fn=load_results,
        outputs=saved_results
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d697dd7c49346ad437.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [9]:
import pandas as pd
from google.colab import files

def analyze_csv(file):
    try:
        df = pd.read_csv(file.name)
        if "text" not in df.columns:
            return "❌ CSV must have a 'text' column."

        sentiments = []
        for t in df["text"]:
            sentiment = classify_sentiment(t)
            sentiments.append(sentiment)

        df["Predicted Sentiment"] = sentiments
        output_file = "sentiment_output.csv"
        df.to_csv(output_file, index=False)
        return df, output_file

    except Exception as e:
        return f"❌ Error: {str(e)}"


In [10]:
from google.colab import files

uploaded_file = files.upload()


Saving sample_legal_texts (2).csv to sample_legal_texts (2).csv


In [11]:
import io

filename = list(uploaded_file.keys())[0]
file_obj = io.BytesIO(uploaded_file[filename])
file_obj.name = filename

result = analyze_csv(file_obj)

if isinstance(result, tuple):
    df_result, output_csv = result
    display(df_result.head())
else:
    print("❌ Error:", result)


,text,Predicted Sentiment
0,The plaintiff agreed to the terms with satisfa...,Positive
1,The contract was violated multiple times witho...,Negative. The document details a breach of con...
2,All parties signed the agreement peacefully.,Neutral
3,The client expressed frustration with the lega...,Negative
4,No further action was deemed necessary by eith...,Neutral


In [12]:
from google.colab import files
files.download(output_csv)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
with gr.Blocks(title="⚖️ AI Legal Sentiment Analyzer") as demo:
    gr.Markdown("# ⚖️ AI Legal Sentiment Analyzer")
    gr.Markdown("Upload or paste legal text to classify sentiment as Positive, Neutral, or Negative.")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="📄 Legal Paragraph", placeholder="Paste your legal paragraph here...", lines=8)
            analyze_button = gr.Button("🔍 Analyze Sentiment")

        with gr.Column():
            sentiment_output = gr.Textbox(label="🧠 Predicted Sentiment", interactive=False)

    with gr.Accordion("📂 Previous Analyses", open=False):
        saved_results = gr.JSON(label="📜 History")
        refresh_btn = gr.Button("🔄 Refresh History")
        refresh_btn.click(fn=load_results, outputs=saved_results)

    analyze_button.click(
        fn=analyze_and_save,
        inputs=text_input,
        outputs=sentiment_output
    ).then(
        fn=load_results,
        outputs=saved_results
    )

    gr.Markdown("---")
    gr.Markdown("## 📤 Bulk Sentiment Analysis via CSV Upload")

    csv_input = gr.File(label="📎 Upload CSV (with 'text' column)", file_types=[".csv"])
    csv_output = gr.File(label="📥 Download Tagged CSV", interactive=False, visible=False)

    def handle_csv(file_obj):
        result = analyze_csv(file_obj)
        if isinstance(result, tuple):
            _, out_file = result
            return gr.update(value=out_file, visible=True)
        return gr.update(visible=False)

    process_csv_btn = gr.Button("📊 Analyze CSV")
    process_csv_btn.click(fn=handle_csv, inputs=csv_input, outputs=csv_output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://67b94959ed6af70472.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
